# **Experiment 2: Feature Engineering and Dimensionality Reduction**

## ***Objectives for this Notebook***

Machine learning models thrive on good data. But raw data often comes messy and unrefined, holding hidden gems amidst irrelevant clutter. This is where feature engineering and dimensionality reduction come in like superheroes, transforming your data from unpolished ore to gleaming treasure.

Feature engineering and dimensionality reduction are two crucial steps in the machine learning pipeline that can significantly improve the performance of your models.

**Feature Engineering** is the process of transforming raw data into features that are suitable for training and deploying machine learning models. Simply, it's about:
* Selecting the right features: Choosing the most relevant features that hold predictive power for your target variable. Think carefully, irrelevant features can mislead your model!
* Creating new features: Combining existing features or extracting hidden patterns to unlock deeper insights. New features can be like secret weapons for your model!
* Transforming features: Scaling, normalizing, or encoding categorical data to ensure all features play fair in the model's eyes. No one wants features dominating the competition due to unfair advantages!


**Dimensionality reduction** is a technique used in machine learning and statistics to reduce the number of features or variables in a dataset while preserving its essential information. The goal is to simplify the data and improve computational efficiency, mitigate the curse of dimensionality, and enhance the performance of machine learning models.

Algorithms:
* Non-Negative Matrix Factorization (NMF): NMF is a factorization technique that decomposes a matrix into two non-negative matrices. It is particularly useful for non-negative data, such as images or text, and is often applied in topic modeling and image processing.
* Latent Dirichlet Allocation (LDA): LDA is a probabilistic generative model used for topic modeling. It assumes that documents are mixtures of topics and that each word's presence is attributable to one of the document's topics. LDA helps discover the underlying topics in a collection of documents.


## **1.) Loading the dataset and setting parameters**

In [116]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import NMF, LatentDirichletAllocation, PCA
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, balanced_accuracy_score

import random
from sklearn.utils import check_random_state

seed_value = 42
# Set Python seed
random.seed(seed_value)

# Set NumPy seed
np.random.seed(seed_value)

# Set scikit-learn seed
sklearn_random_state = check_random_state(seed_value)

In [2]:
microbiome_df = pd.read_csv("./dataset/microbiome_preprocessed_files/microbiome_merged_dfs.csv")
microbiome_df

,name,Simonsiella,Treponema,Campylobacter,Helicobacter,Paracoccus,Comamonas,Pseudomonas,Xanthomonas,Agrobacterium,...,Hungatella,Pseudopropionibacterium,Peptoanaerobacter,Emergencia,Prevotellamassilia,Criibacterium,Fournierella,Negativibacillus,Duodenibacillus,label
0,TCGA-CG-5720-01A,0.0,0.000000,0.000000,0.895050,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,STAD
1,TCGA-CN-4741-01A,0.0,0.000000,0.010470,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,HNSC
2,TCGA-BR-6801-01A,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,STAD
3,TCGA-IG-A3I8-01A,0.0,0.000000,0.000000,0.067717,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,ESCA
4,TCGA-L5-A4OT-01A,0.0,0.000000,0.012202,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,ESCA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,TCGA-CG-5719-01A,0.0,0.000000,0.000000,0.106557,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,STAD
508,TCGA-CQ-5329-01A,0.0,0.175564,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.136613,0.0,0.0,0.0,0.0,0.0,0.0,HNSC
509,TCGA-CQ-7068-01A,0.0,0.335060,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.011534,0.0,0.0,0.0,0.0,0.0,0.0,HNSC
510,TCGA-CG-4455-01A,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.014781,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,STAD


In [10]:
classes = ["HNSC", "STAD", "COAD", "ESCA", "READ"]

## **2.) Creating a data loader**

In [46]:
exp1_best_hyperparam_report = pd.read_csv("./dataset/microbiome_preprocessed_files/exp1_best_hyperparam.csv")

,label,criterion,max_depth,min_samples_leaf,min_samples_split,n_estimators,test_score,mean_train_score,mean_validation_score
0,HNSC,entropy,20,1,40,50,0.785829,0.911599,0.831183
1,STAD,entropy,30,1,10,100,0.780853,0.981524,0.793080
2,COAD,gini,20,1,10,200,0.895644,0.965040,0.892741
3,ESCA,gini,10,1,10,1,0.596405,0.734715,0.669170
4,READ,entropy,10,1,20,1,0.700000,0.676505,0.609679


In [120]:
# Function to create hyperparameters, we use the best hyperparameters for the rf classifier from experiment 1

def create_hyperparameter_grids(report_df, additional_param=None):
    new_hyperparamter_grid_by_class = {}

    # Read the best hyperparameters from experiment 1
    exclude_cols = ["label", "test_score", "mean_train_score", "mean_validation_score"] # We don't want these columns to be in our future grid parameters
    if report_df is None or report_df.empty:
        raise ValueError("Report DataFrame is required.")

    for c in report_df.index: 
        # Choose a row (class) from the report DataFrame
        selected_class = report_df.index[c]
        # Get the best hyperparameters and remove excluded columns
        exp1_best_hyperparameters = report_df.loc[selected_class].drop(exclude_cols).to_dict()

        exp1_best_hyperparameters = {f"{key}": [value] for key, value in exp1_best_hyperparameters.items()} # The format we want for classifier hyperparameters

        if additional_param != None:
            # Add the additional parameters
            exp1_best_hyperparameters = {f"rf__{key}": value for key, value in exp1_best_hyperparameters.items()} #Adjust prefix of hyperparameters
            combined_param_grid = {**exp1_best_hyperparameters, **additional_param}
            new_hyperparamter_grid_by_class[report_df.loc[selected_class]["label"]] = combined_param_grid
        else:
            new_hyperparamter_grid_by_class[report_df.loc[selected_class]["label"]] = exp1_best_hyperparameters

    return new_hyperparamter_grid_by_class

nmf_pca_param_grid = {
    "transformer__n_components": list(range(4,65)) # In paper, features 4 to 64
}

lda_param_grid = {
    "transformer__n_components": list(range(1, len(classes))) # In paper, features 1 to length of class-1
}

In [121]:
def augment_dataset(X,y,hyperparameters,algorithm="NMF"):

    augmented_datasets_dict = {}
    
    # Create the pipeline object. Here we are chaining the dimensionality reduction step and the classifier step.

    algorithms_dict = {"NMF": NMF(), "LDA": LatentDirichletAllocation(), "PCA": PCA()}
    pipeline = Pipeline([
        ('transformer', algorithms_dict[algorithm]),
        ('rf', RandomForestClassifier(random_state=seed_value))
    ])

    # hyperparameters
    combined_param_grid = create_hyperparameter_grids(hyperparameters,lda_param_grid if algorithm=="LDA" else nmf_pca_param_grid)
 
    print(f'Using algorithm {algorithm}:')
    for i in classes:
        print(f"Class {i}")
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed_value)

        # Create the GridSearchCV object
        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=combined_param_grid[i],
            scoring=make_scorer(balanced_accuracy_score),  # Choose an appropriate metric for your problem
            cv=cv,
            n_jobs=-1,  # Use all available processors
            return_train_score=True  # Set to True to calculate train scores
        )

        # Perform the grid search
        print("Fitting..")
        grid_search.fit(X, y)

        # Print the best hyperparameters
        print("Best Hyperparameters:", grid_search.best_params_)

        # Get the best model
        best_model = grid_search.best_estimator_

        # LENGTH OF DATASET SHOULD STAY SIMILAR TO ORIGINAL, but length of features should increase by the best n_components

        # Transform the original dataset to get the latent features
        latent_features = best_model.named_steps['transformer'].transform(X)
        print(f"Count of new columns/latent features: {len(latent_features.T)}")

        print(f"Count of columns of original feature set: {len(X.T)}")

        feature_augmented = pd.concat((X, pd.DataFrame(latent_features, index=X.index, columns=[f"new_feature {str(x)}" for x in range(1,len(latent_features.T)+1)])), axis=1)
        print(f"Count of columns of augmented feature set: {len(feature_augmented.T)}")
        print(f"Count of rows of augmented dataset: {len(feature_augmented)}")

        augmented_datasets_dict[i] = feature_augmented

    return augmented_datasets_dict




In [122]:
def exp_2_data_loader(dataframe, 
                         label_column, 
                         classes, 
                         report_df=exp1_best_hyperparam_report, 
                         algorithm="NMF", 
                         train_test=True):
    """

    Returns:
    - dataset_dict: a dictionary where the keys is the targeted class and the values are its corresponding features and labels
    """
    augmented_dataset_dict = {}

    exclude_cols = ["name", "label"]

    feat = dataframe.drop(columns=exclude_cols)
    tar = dataframe["label"]

    # print(f"len of feat: {len(feat)}")
    # print(f"len of tar: {len(tar)}")

    augmented_features_dict = augment_dataset(feat, tar,hyperparameters=report_df, algorithm=algorithm)


    # print(f"Count of rows in original dataset:{len(dataframe)}")
    # print(f"Count of rows in augmented dataset:{len(feature_augmented)}")

    for i in classes:
        positive_class = i
        dframe = pd.DataFrame(index=augmented_features_dict[i].index)
        dframe["name"] = dataframe["name"]
        dframe = pd.concat([dframe, augmented_features_dict[i]], axis=1)
        dframe['label'] = [1 if x == positive_class else 0 for x in dataframe[label_column]]
        print(dframe.label.value_counts())
        X = dframe.drop(["name", "label"], axis=1)
        y = dframe["label"]
        if train_test:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, random_state=seed_value)
            augmented_dataset_dict[positive_class] = {"train": (X_train, y_train),
                                            "test": (X_test, y_test)}
        else:
            augmented_dataset_dict[positive_class] = {"feature": X, 
                                            "label": y}

    return augmented_dataset_dict

In [123]:
exp2_datasets_nmf = exp_2_data_loader(microbiome_df, "label", classes=classes, report_df=exp1_best_hyperparam_report, algorithm="NMF", train_test=True)

Using algorithm NMF:
Class HNSC
Fitting..
Best Hyperparameters: {'rf__criterion': 'entropy', 'rf__max_depth': 20, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 40, 'rf__n_estimators': 50, 'transformer__n_components': 15}
Count of new columns/latent features: 15
Count of columns of original feature set: 131
Count of columns of augmented feature set: 146
Count of rows of augmented dataset: 512
Class STAD
Fitting..
Best Hyperparameters: {'rf__criterion': 'entropy', 'rf__max_depth': 30, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 10, 'rf__n_estimators': 100, 'transformer__n_components': 27}
Count of new columns/latent features: 27
Count of columns of original feature set: 131
Count of columns of augmented feature set: 158
Count of rows of augmented dataset: 512
Class COAD
Fitting..
Best Hyperparameters: {'rf__criterion': 'gini', 'rf__max_depth': 20, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 10, 'rf__n_estimators': 200, 'transformer__n_components': 26}
Count of new c

In [124]:
exp2_datasets_lda = exp_2_data_loader(microbiome_df, "label", classes=classes, report_df=exp1_best_hyperparam_report, algorithm="LDA", train_test=True)

Using algorithm LDA:
Class HNSC
Fitting..
Best Hyperparameters: {'rf__criterion': 'entropy', 'rf__max_depth': 20, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 40, 'rf__n_estimators': 50, 'transformer__n_components': 3}
Count of new columns/latent features: 3
Count of columns of original feature set: 131
Count of columns of augmented feature set: 134
Count of rows of augmented dataset: 512
Class STAD
Fitting..
Best Hyperparameters: {'rf__criterion': 'entropy', 'rf__max_depth': 30, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 10, 'rf__n_estimators': 100, 'transformer__n_components': 4}
Count of new columns/latent features: 4
Count of columns of original feature set: 131
Count of columns of augmented feature set: 135
Count of rows of augmented dataset: 512
Class COAD
Fitting..
Best Hyperparameters: {'rf__criterion': 'gini', 'rf__max_depth': 20, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 10, 'rf__n_estimators': 200, 'transformer__n_components': 4}
Count of new column

In [125]:
exp2_datasets_pca = exp_2_data_loader(microbiome_df, "label", classes=classes, report_df=exp1_best_hyperparam_report, algorithm="PCA", train_test=True)

Using algorithm PCA:
Class HNSC
Fitting..
Best Hyperparameters: {'rf__criterion': 'entropy', 'rf__max_depth': 20, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 40, 'rf__n_estimators': 50, 'transformer__n_components': 43}
Count of new columns/latent features: 43
Count of columns of original feature set: 131
Count of columns of augmented feature set: 174
Count of rows of augmented dataset: 512
Class STAD
Fitting..
Best Hyperparameters: {'rf__criterion': 'entropy', 'rf__max_depth': 30, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 10, 'rf__n_estimators': 100, 'transformer__n_components': 44}
Count of new columns/latent features: 44
Count of columns of original feature set: 131
Count of columns of augmented feature set: 175
Count of rows of augmented dataset: 512
Class COAD
Fitting..
Best Hyperparameters: {'rf__criterion': 'gini', 'rf__max_depth': 20, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 10, 'rf__n_estimators': 200, 'transformer__n_components': 64}
Count of new c

In [107]:
exp2_datasets_nmf["HNSC"]["train"][0]

,Simonsiella,Treponema,Campylobacter,Helicobacter,Paracoccus,Comamonas,Pseudomonas,Xanthomonas,Agrobacterium,Bradyrhizobium,...,new_feature 6,new_feature 7,new_feature 8,new_feature 9,new_feature 10,new_feature 11,new_feature 12,new_feature 13,new_feature 14,new_feature 15
155,0.0,0.010944,0.0,0.000000,0.0,0.0,0.017489,0.0,0.0,0.0,...,0.000091,0.019054,0.032869,0.006719,0.000503,0.000111,0.001004,0.002378,0.021345,0.017670
414,0.0,0.000000,0.0,0.107591,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.072503,0.017535,0.028632,0.001330,0.000000,0.000024,0.000130,0.000000,0.048651,0.000000
172,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000921,0.000000,0.002354,0.000000,0.000996
367,0.0,0.000000,0.0,0.421787,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.284333,0.020627,0.059244,0.000000,0.000000,0.000000,0.002250,0.000000,0.035827,0.000000
462,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000325,0.000000,0.000000,0.000179,0.000106,0.000000,0.000000,0.000830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.017618,0.000873,0.000000,0.000574,0.000129,0.000138,0.000000,0.000000,0.000064
15,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000312,0.000199,0.000822,0.000000,0.004203,0.000000,0.000775,0.014377,0.000000,0.000000
198,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.029941,0.000236,0.000000,0.000000,0.000000,0.000000,0.000000,0.011949,0.000499
211,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000144,0.000000,0.000000,0.010166,0.000055,0.000139,0.000000,0.000423


In [108]:
exp1_best_hyperparam_by_class = create_hyperparameter_grids(exp1_best_hyperparam_report, None)
exp1_best_hyperparam_report

,label,criterion,max_depth,min_samples_leaf,min_samples_split,n_estimators,test_score,mean_train_score,mean_validation_score
0,HNSC,entropy,20,1,40,50,0.785829,0.911599,0.831183
1,STAD,entropy,30,1,10,100,0.780853,0.981524,0.793080
2,COAD,gini,20,1,10,200,0.895644,0.965040,0.892741
3,ESCA,gini,10,1,10,1,0.596405,0.734715,0.669170
4,READ,entropy,10,1,20,1,0.700000,0.676505,0.609679


In [109]:
def perform_gridsearchcv(dataset_dict, classes, cv_n_splits=5, n_jobs=12):
    report = {}
    for c in classes:
        print("Class: ", c)
        features = dataset_dict[c]["train"][0]
        target = dataset_dict[c]["train"][1]

        # Define the classifier
        rf = RandomForestClassifier(random_state=seed_value)

        exp1_best_hyperparameter_grids = create_hyperparameter_grids(exp1_best_hyperparam_report, None)

        cv = StratifiedKFold(n_splits=cv_n_splits, shuffle=True, random_state=seed_value)

        # Create the GridSearchCV object
        grid_search = GridSearchCV(
            estimator=rf,
            param_grid=exp1_best_hyperparameter_grids[c],
            scoring=make_scorer(balanced_accuracy_score),  # Choose an appropriate metric for your problem
            cv=cv,
            n_jobs=-1,  # Use all available processors,
            return_train_score=True
)

        # Perform the grid search
        grid_search.fit(features, target)

        # Access cv_results_ attribute to get detailed results
        cv_results = grid_search.cv_results_

        # Print the best hyperparameters
        print("Best Hyperparameters:", grid_search.best_params_)

        # Get the best model
        best_model = grid_search.best_estimator_

        # Evaluate the best model on your test set
        # Assuming you have X_test and y_test from your data
        # Modify this based on your actual test set
        predictions = best_model.predict(dataset_dict[c]["test"][0])
        accuracy = balanced_accuracy_score(dataset_dict[c]["test"][1], predictions)
        print("Accuracy on Test Set:", accuracy)


        # Calculate mean train score and mean test score
        mean_train_score = cv_results['mean_train_score'][grid_search.best_index_]
        mean_test_score = cv_results['mean_test_score'][grid_search.best_index_]

        print("Mean Train Score:", mean_train_score)
        print("Mean Test Score:", mean_test_score)


        report[c] = {
            "model": best_model,
            "best_hyperparameters": grid_search.best_params_,
            "test_score": accuracy,
            "mean_train_score": mean_train_score,
            "mean_validation_score": mean_test_score
        }
        
    return report


In [110]:
exp2_nmf_report = perform_gridsearchcv(exp2_datasets_nmf, classes)
exp2_nmf_report

Class:  HNSC
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 40, 'n_estimators': 50}
Accuracy on Test Set: 0.7455716586151369
Mean Train Score: 0.8906344127746835
Mean Test Score: 0.7867037971136333
Class:  STAD
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Accuracy on Test Set: 0.780852994555354
Mean Train Score: 0.9718924060663319
Mean Test Score: 0.7768231768231768
Class:  COAD
Best Hyperparameters: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Accuracy on Test Set: 0.8870235934664247
Mean Train Score: 0.9740275595201787
Mean Test Score: 0.8702530802530802
Class:  ESCA
Best Hyperparameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 1}
Accuracy on Test Set: 0.5890522875816994
Mean Train Score: 0.7551205454194649
Mean T

{'HNSC': {'model': RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_split=40,
                         n_estimators=50, random_state=42),
  'best_hyperparameters': {'criterion': 'entropy',
   'max_depth': 20,
   'min_samples_leaf': 1,
   'min_samples_split': 40,
   'n_estimators': 50},
  'test_score': 0.7455716586151369,
  'mean_train_score': 0.8906344127746835,
  'mean_validation_score': 0.7867037971136333},
 'STAD': {'model': RandomForestClassifier(criterion='entropy', max_depth=30, min_samples_split=10,
                         random_state=42),
  'best_hyperparameters': {'criterion': 'entropy',
   'max_depth': 30,
   'min_samples_leaf': 1,
   'min_samples_split': 10,
   'n_estimators': 100},
  'test_score': 0.780852994555354,
  'mean_train_score': 0.9718924060663319,
  'mean_validation_score': 0.7768231768231768},
 'COAD': {'model': RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=200,
                         random_state=42),
  'best_hy

In [87]:
def best_hyperparameter_to_df(nested_dict, exp_name):
    path = "./dataset/microbiome_preprocessed_files/"
    data = []

    for key, value in nested_dict.items():
        entry = {'label': key}
        entry.update(value['best_hyperparameters'])
        entry['test_score'] = value['test_score']
        entry['mean_train_score'] = value['mean_train_score']
        entry['mean_validation_score'] = value['mean_validation_score']
        data.append(entry)

    df = pd.DataFrame(data)

    df.to_csv(path+f"{exp_name}_best_hyperparam.csv", index=False)
    return df

In [88]:
best_hyperparameter_to_df(exp2_nmf_report, "exp2_nmf")

,label,criterion,max_depth,min_samples_leaf,min_samples_split,n_estimators,test_score,mean_train_score,mean_validation_score
0,HNSC,entropy,20,1,40,50,0.745572,0.890634,0.786704
1,STAD,entropy,30,1,10,100,0.780853,0.971892,0.776823
2,COAD,gini,20,1,10,200,0.887024,0.974028,0.870253
3,ESCA,gini,10,1,10,1,0.518791,0.739436,0.632980
4,READ,entropy,10,1,20,1,0.557143,0.670943,0.589695


In [90]:
exp2_lda_report = perform_gridsearchcv(exp2_datasets_lda, classes)
exp2_lda_report

Class:  HNSC
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 40, 'n_estimators': 50}
Accuracy on Test Set: 0.7548309178743962
Mean Train Score: 0.8978200475170173
Mean Test Score: 0.8194435103451496
Class:  STAD
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Accuracy on Test Set: 0.7722323049001816
Mean Train Score: 0.9711127571311096
Mean Test Score: 0.785001665001665
Class:  COAD
Best Hyperparameters: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Accuracy on Test Set: 0.8784029038112522
Mean Train Score: 0.9736985634212207
Mean Test Score: 0.8914418914418913
Class:  ESCA
Best Hyperparameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 1}
Accuracy on Test Set: 0.6299019607843137
Mean Train Score: 0.711817057178144
Mean Te

{'HNSC': {'model': RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_split=40,
                         n_estimators=50, random_state=42),
  'best_hyperparameters': {'criterion': 'entropy',
   'max_depth': 20,
   'min_samples_leaf': 1,
   'min_samples_split': 40,
   'n_estimators': 50},
  'test_score': 0.7548309178743962,
  'mean_train_score': 0.8978200475170173,
  'mean_validation_score': 0.8194435103451496},
 'STAD': {'model': RandomForestClassifier(criterion='entropy', max_depth=30, min_samples_split=10,
                         random_state=42),
  'best_hyperparameters': {'criterion': 'entropy',
   'max_depth': 30,
   'min_samples_leaf': 1,
   'min_samples_split': 10,
   'n_estimators': 100},
  'test_score': 0.7722323049001816,
  'mean_train_score': 0.9711127571311096,
  'mean_validation_score': 0.785001665001665},
 'COAD': {'model': RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=200,
                         random_state=42),
  'best_hy

In [91]:
best_hyperparameter_to_df(exp2_lda_report, "exp2_lda")

,label,criterion,max_depth,min_samples_leaf,min_samples_split,n_estimators,test_score,mean_train_score,mean_validation_score
0,HNSC,entropy,20,1,40,50,0.754831,0.897820,0.819444
1,STAD,entropy,30,1,10,100,0.772232,0.971113,0.785002
2,COAD,gini,20,1,10,200,0.878403,0.973699,0.891442
3,ESCA,gini,10,1,10,1,0.629902,0.711817,0.593534
4,READ,entropy,10,1,20,1,0.557143,0.640717,0.552925


In [126]:
exp2_pca_report = perform_gridsearchcv(exp2_datasets_pca, classes)
exp2_pca_report

Class:  HNSC
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 40, 'n_estimators': 50}
Accuracy on Test Set: 0.7673107890499196
Mean Train Score: 0.9136923226841276
Mean Test Score: 0.7605202933071786
Class:  STAD
Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Accuracy on Test Set: 0.7282214156079855
Mean Train Score: 0.9869165896370118
Mean Test Score: 0.7227805527805528
Class:  COAD
Best Hyperparameters: {'criterion': 'gini', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Accuracy on Test Set: 0.8607078039927405
Mean Train Score: 0.976613263348893
Mean Test Score: 0.8694105894105896
Class:  ESCA
Best Hyperparameters: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 1}
Accuracy on Test Set: 0.4893790849673203
Mean Train Score: 0.7855913929956593
Mean T

{'HNSC': {'model': RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_split=40,
                         n_estimators=50, random_state=42),
  'best_hyperparameters': {'criterion': 'entropy',
   'max_depth': 20,
   'min_samples_leaf': 1,
   'min_samples_split': 40,
   'n_estimators': 50},
  'test_score': 0.7673107890499196,
  'mean_train_score': 0.9136923226841276,
  'mean_validation_score': 0.7605202933071786},
 'STAD': {'model': RandomForestClassifier(criterion='entropy', max_depth=30, min_samples_split=10,
                         random_state=42),
  'best_hyperparameters': {'criterion': 'entropy',
   'max_depth': 30,
   'min_samples_leaf': 1,
   'min_samples_split': 10,
   'n_estimators': 100},
  'test_score': 0.7282214156079855,
  'mean_train_score': 0.9869165896370118,
  'mean_validation_score': 0.7227805527805528},
 'COAD': {'model': RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=200,
                         random_state=42),
  'best_h